In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
from IPython.display import YouTubeVideo
import matplotlib.pyplot as plt

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
print(check_output(["ls", "../input/video_level"]).decode("utf8"))


In [ ]:
frame_lvl_record = "../input/frame_level/train-1.tfrecord"

def rgb_and_audio_from(tf_seq_example):
    feat_rgb = []
    feat_audio = []
    n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
    sess = tf.InteractiveSession()
    rgb_frame = []
    audio_frame = []
    # iterate through frames for that example
    for i in range(n_frames):
        rgb_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        audio_frame.append(tf.cast(tf.decode_raw(
                tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                       ,tf.float32).eval())
        
        
    sess.close()
    feat_rgb.append(rgb_frame)
    feat_audio.append(audio_frame)
    return feat_rgb, feat_audio

# now, let's read the frame-level data
# due to execution time, we're only going to read the first video

i = 0
for example in tf.python_io.tf_record_iterator(frame_lvl_record):     
#    i+=1
#    if i < 3:
#        continue
    tf_seq_example = tf.train.SequenceExample.FromString(example)
    video_id = tf_seq_example.context.feature['video_id'].bytes_list.value[0]
    print('https://www.youtube.com/watch?v={}'.format(str(video_id)))
    rgb, audio = rgb_and_audio_from(tf_seq_example)
    break

In [ ]:
filenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]
labels_df = pd.read_csv('../input/label_names.csv')

video_lables = []
video_rgb_mean = []
video_audio_mean = []
labels = []

for filename in filenames:
    print(filename)
    i = 0;
    for example in tf.python_io.tf_record_iterator(filename):
        tf_example = tf.train.Example.FromString(example)
        label_example = list(tf_example.features.feature['labels'].int64_list.value)
        label_example_textual = list(labels_df[labels_df['label_id'].isin(label_example)]['label_name'])
        int_labels = []
        for k in range(0, len(label_example_textual)):
            v = labels_df[labels_df['label_name'] == label_example_textual[k]].index
            int_labels.append(v[0])
        rgb_mean = tf_example.features.feature['mean_rgb'].float_list.value
        audio_mean = tf_example.features.feature['mean_rgb'].float_list.value
        video_rgb_mean.append(rgb_mean)
        video_audio_mean.append(audio_mean)
        labels.append(int_labels)
        if((i % 100000) == 0):
            print(i)
        i = i + 1
X_list = video_audio_mean
Y = labels

In [ ]:
X = np.array(X_list)
print(len(labels))
new_label = []
for i in range(0, len(Y)):
    #if(i % 10 == 0):
    #    print(i)
    if(len(Y[i]) == 0):
        new_label.append(0)
    else:
        new_label.append(Y[i][0])
y = np.array(new_label)
y.shape
labels[1]
X.shape

In [ ]:
len(y[3390])

In [ ]:
## importing the required packages
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
             discriminant_analysis, random_projection)
## Loading and curating the data
#digits = datasets.load_digits(n_class=10)
#X = digits.data
#y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30
## Function to Scale and visualize the embedding vectors
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)     
    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 4716.),
                 fontdict={'weight': 'bold', 'size': 4716})
    if hasattr(offsetbox, 'AnnotationBbox'):
        ## only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(y.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                ## don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(y[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

#----------------------------------------------------------------------
plt.title('A selection from the 64-dimensional digits dataset')
## Computing PCA
print("Computing PCA projection")
t0 = time()
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))
## Computing t-SNE
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)
plot_embedding(X_tsne,
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))
plt.show()

In [ ]:
v = labels_df[labels_df['label_name'] == 'Car'].index
v[0]

In [ ]:
v.at[0, 'label_id']